In [1]:
# Import libraries for crawling
import requests
from bs4 import BeautifulSoup
import numpy as np

# Import PySpark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType

## Creates spark session
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName('Dota2 Crawler').getOrCreate()

23/04/07 20:51:29 WARN Utils: Your hostname, DESKTOP-L7KNHUE resolves to a loopback address: 127.0.1.1; using 192.168.45.127 instead (on interface eth0)
23/04/07 20:51:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/07 20:51:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
# Creates dataframe with schema for the dataset
emptyRDD = spark.sparkContext.emptyRDD()

## Create schema
schema = StructType([
  StructField('hero', StringType(), True),
  StructField('header', StringType(), True),
  StructField('type', StringType(), True),
  StructField('counter', StringType(), True)
  ])

# Creates temp dataframe
df = spark.createDataFrame(emptyRDD,schema)
df.printSchema()

root
 |-- hero: string (nullable = true)
 |-- header: string (nullable = true)
 |-- type: string (nullable = true)
 |-- counter: string (nullable = true)



In [3]:
page = requests.get('https://dota2.fandom.com/wiki/Abaddon/Counters')
soup = BeautifulSoup(page.text, 'html.parser')

In [14]:
# Pull all text from body div
hero_name_list = soup.find_all(style='font-size:12pt;display:inline;')
header_list = soup.find_all(class_='mw-headline')
item_list = soup.find_all(class_='image-link')

# user-defined function for creating nested dictionary
def dotaDict(scrape_list):
    text = []
    srcline = []
    for h in scrape_list:
        text.append(h.get_text())
        srcline.append(h.sourceline)
    arr = np.column_stack((text,srcline))
    return arr

# created nested dictionary
## header
headers = dotaDict(header_list)
## heroes
heroes = dotaDict(hero_name_list)
## items
items = dotaDict(item_list)

In [25]:
headers[2][0]

'Items'

In [26]:
heroes

array([['Ancient Apparition', '1334'],
       ['Axe', '1337'],
       ['Brewmaster', '1340'],
       ['Doom', '1344'],
       ['Lina', '1347'],
       ['Outworld Destroyer', '1350'],
       ['Shadow Demon', '1354'],
       ['Death Prophet', '1369'],
       ['Mirana', '1373'],
       ['Bane', '1376'],
       ['Batrider', '1380'],
       ['Beastmaster', '1384'],
       ['Chen', '1388'],
       ['Techies', '1391'],
       ['Bloodseeker', '1396'],
       ['Necrophos', '1399'],
       ['Legion Commander', '1402'],
       ['Nyx Assassin', '1405'],
       ['Storm Spirit', '1419'],
       ['Phantom Assassin', '1422'],
       ['Io', '1425'],
       ['Axe', '1428'],
       ['Legion Commander', '1431'],
       ['Centaur Warrunner', '1434'],
       ['Oracle', '1437']], dtype='<U21')